In [26]:
import requests
#import mysql.connector
import os
#from dotenv import load_dotenv
#import json
#import pandas as pd
import pymysql
import time
import cryptography

# Configuración API
URL = "https://cramer.buk.cl/api/v1/chile/employees"
TOKEN = "Xegy8dVsa1H8SFfojJcwYtDL"


# Configuración BD
DB_HOST = "10.254.33.62"
DB_USER = "compensaciones_rrhh"
DB_PASSWORD = "_Cramercomp2025_"
DB_NAME = "rrhh_app"

# Imprimir las variables de entorno
# print(f"TOKEN: {os.getenv('BUK_AUTH_TOKEN')}")
# print(f"SQL: {os.getenv('clave_sql')}")


In [ ]:
def obtener_todos_los_empleados_filtrados():
    """
    Obtiene todos los empleados desde la API con paginación y los devuelve filtrados
    con solo los campos necesarios.
    """
    headers = {"auth_token": TOKEN}
    empleados_filtrados = []
    url_actual = URL
    pagina_actual = 1
    
    print("🚀 Comenzando la obtención de todos los empleados con paginación...")
    
    while url_actual:
        print(f"📄 Obteniendo página {pagina_actual}...")
        
        try:
            respuesta = requests.get(url_actual, headers=headers)
            respuesta.raise_for_status()
            
            respuesta_api = respuesta.json()
            empleados_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']
            
            # Filtrar cada empleado de esta página
            for empleado_completo in empleados_pagina:
                empleado_filtrado = {
                    "person_id": empleado_completo.get("person_id"),
                    "id": empleado_completo.get("id"),
                    "full_name": empleado_completo.get("full_name"),
                    "rut": empleado_completo.get("rut"),
                    "email": empleado_completo.get("email"),
                    "personal_email": empleado_completo.get("personal_email"),
                    "active_since": empleado_completo.get("active_since"),
                    "status": empleado_completo.get("status"),
                    "payment_method": empleado_completo.get("payment_method"),
                    "id_boss": empleado_completo.get("current_job", {}).get("boss", {}).get("id"),
                    "rut_boss": empleado_completo.get("current_job", {}).get("boss", {}).get("rut"),
                    "contract_type": empleado_completo.get("current_job", {}).get("contract_type"),
                    "start_date": empleado_completo.get("current_job", {}).get("start_date"),
                    "end_date": empleado_completo.get("current_job", {}).get("end_date"),
                    "contract_finishing_date_1": empleado_completo.get("current_job", {}).get("contract_finishing_date_1"),
                    "contract_finishing_date_2": empleado_completo.get("current_job", {}).get("contract_finishing_date_2"),
                    "address": empleado_completo.get("address"),
                    "street": empleado_completo.get("street"),
                    "street_number": empleado_completo.get("street_number"),
                    "city": empleado_completo.get("city"),
                    # AGREGA ESTE CAMPO FALTANTE
                    "province": empleado_completo.get("province"),  # <-- Este campo es el que falta
                    "district": empleado_completo.get("district"),
                    "region": empleado_completo.get("region"),
                    "phone": empleado_completo.get("phone"),
                    "gender": empleado_completo.get("gender"),
                    "birthday": empleado_completo.get("birthday"),
                    "university": empleado_completo.get("university"),
                    "degree": empleado_completo.get("degree"),
                    "bank": empleado_completo.get("bank"),
                    "account_type": empleado_completo.get("account_type"),
                    "account_number": empleado_completo.get("account_number"),
                    "nationality": empleado_completo.get("nationality"),
                    "civil_status": empleado_completo.get("civil_status"),
                    "health_company": empleado_completo.get("health_company"),
                    "pension_regime": empleado_completo.get("pension_regime"),
                    "pension_fund": empleado_completo.get("pension_fund"),
                    "active_until": empleado_completo.get("active_until"),
                    "termination_reason": empleado_completo.get("termination_reason"),
                }
                empleados_filtrados.append(empleado_filtrado)
            
            print(f"✅ Página {pagina_actual}: {len(empleados_pagina)} empleados procesados")
            print(f"📊 Total acumulado: {len(empleados_filtrados)} empleados filtrados")
            
            # Obtener la URL de la siguiente página
            url_actual = pagination_info.get('next')
            pagina_actual += 1
            
            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la petición: {e}")
            break
            
    print(f"🎉 ¡Paginación completada! Total de empleados filtrados: {len(empleados_filtrados)}")
    return empleados_filtrados

# Usar la función híbrida
empleados_filtrados = obtener_todos_los_empleados_filtrados()

🚀 Comenzando la obtención de todos los empleados con paginación...
📄 Obteniendo página 1...
✅ Página 1: 25 empleados procesados
📊 Total acumulado: 25 empleados filtrados
✅ Página 1: 25 empleados procesados
📊 Total acumulado: 25 empleados filtrados
📄 Obteniendo página 2...
📄 Obteniendo página 2...
✅ Página 2: 25 empleados procesados
📊 Total acumulado: 50 empleados filtrados
✅ Página 2: 25 empleados procesados
📊 Total acumulado: 50 empleados filtrados
📄 Obteniendo página 3...
📄 Obteniendo página 3...
✅ Página 3: 25 empleados procesados
📊 Total acumulado: 75 empleados filtrados
✅ Página 3: 25 empleados procesados
📊 Total acumulado: 75 empleados filtrados
📄 Obteniendo página 4...
📄 Obteniendo página 4...
✅ Página 4: 25 empleados procesados
📊 Total acumulado: 100 empleados filtrados
✅ Página 4: 25 empleados procesados
📊 Total acumulado: 100 empleados filtrados
📄 Obteniendo página 5...
📄 Obteniendo página 5...
✅ Página 5: 25 empleados procesados
📊 Total acumulado: 125 empleados filtrados
✅ P

In [36]:
print("🚀 Conectando a MySQL...")
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    #charset='utf8mb4'
)
cursor = conexion.cursor()

# Crear base de datos si no existe
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")

print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")

print("🚀 Creando tabla employees completa si no existe...")
sql_create_table = """
CREATE TABLE IF NOT EXISTS employees (
    person_id INT,
    id INT PRIMARY KEY,
    full_name VARCHAR(255),
    rut VARCHAR(50),
    email VARCHAR(255),
    personal_email VARCHAR(255),
    active_since DATE,
    status VARCHAR(100),
    payment_method VARCHAR(100),
    id_boss INT,
    rut_boss VARCHAR(50),
    contract_type VARCHAR(100),
    start_date DATE,
    end_date DATE,
    contract_finishing_date_1 DATE,
    contract_finishing_date_2 DATE,
    address TEXT,
    street VARCHAR(255),
    street_number VARCHAR(50),
    city VARCHAR(100),
    province VARCHAR(100),  -- <-- Agregado campo province
    district VARCHAR(100),
    region VARCHAR(100),
    phone VARCHAR(50),
    gender VARCHAR(20),
    birthday DATE,
    university VARCHAR(255),
    degree VARCHAR(255),
    bank VARCHAR(100),
    account_type VARCHAR(100),
    account_number VARCHAR(100),
    nationality VARCHAR(100),
    civil_status VARCHAR(100),
    health_company VARCHAR(100),
    pension_regime VARCHAR(100),
    pension_fund VARCHAR(100),
    active_until DATE,
    termination_reason TEXT
);
"""
cursor.execute(sql_create_table)
print("✅ Tabla 'empleados' creada exitosamente")

🚀 Conectando a MySQL...
✅ Conectado a MySQL y usando la base: rrhh_app
🚀 Creando tabla employees completa si no existe...
✅ Tabla 'empleados' creada exitosamente


In [ ]:
print("🚀 Insertando TODOS los empleados en la tabla SQL...")
contador = 0
errores = 0

for e in empleados_filtrados:  # Usar empleados_filtrados de la función híbrida
    try:
        sql = """
        INSERT INTO employees (
            person_id, id, full_name, rut, email, personal_email, active_since, 
            status, payment_method, id_boss, rut_boss, contract_type, start_date, 
            end_date, contract_finishing_date_1, contract_finishing_date_2, 
            address, street, street_number, city, province, district, region, phone, 
            gender, birthday, university, degree, bank, account_type, 
            account_number, nationality, civil_status, health_company, 
            pension_regime, pension_fund, active_until, termination_reason
        )
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s
        )
        ON DUPLICATE KEY UPDATE
            person_id=VALUES(person_id),
            full_name=VALUES(full_name),
            rut=VALUES(rut),
            email=VALUES(email),
            personal_email=VALUES(personal_email),
            active_since=VALUES(active_since),
            status=VALUES(status),
            payment_method=VALUES(payment_method),
            id_boss=VALUES(id_boss),
            rut_boss=VALUES(rut_boss),
            contract_type=VALUES(contract_type),
            start_date=VALUES(start_date),
            end_date=VALUES(end_date),
            contract_finishing_date_1=VALUES(contract_finishing_date_1),
            contract_finishing_date_2=VALUES(contract_finishing_date_2),
            address=VALUES(address),
            street=VALUES(street),
            street_number=VALUES(street_number),
            city=VALUES(city),
            province=VALUES(province),  -- <-- Agregado campo province
            district=VALUES(district),
            region=VALUES(region),
            phone=VALUES(phone),
            gender=VALUES(gender),
            birthday=VALUES(birthday),
            university=VALUES(university),
            degree=VALUES(degree),
            bank=VALUES(bank),
            account_type=VALUES(account_type),
            account_number=VALUES(account_number),
            nationality=VALUES(nationality),
            civil_status=VALUES(civil_status),
            health_company=VALUES(health_company),
            pension_regime=VALUES(pension_regime),
            pension_fund=VALUES(pension_fund),
            active_until=VALUES(active_until),
            termination_reason=VALUES(termination_reason)
        """
        
        values = (
            e.get("person_id"),
            e.get("id"),
            e.get("full_name"),
            e.get("rut"),
            e.get("email"),
            e.get("personal_email"),
            e.get("active_since"),
            e.get("status"),
            e.get("payment_method"),
            e.get("id_boss"),
            e.get("rut_boss"),
            e.get("contract_type"),
            e.get("start_date"),
            e.get("end_date"),
            e.get("contract_finishing_date_1"),
            e.get("contract_finishing_date_2"),
            e.get("address"),
            e.get("street"),
            e.get("street_number"),
            e.get("city"),
            # AGREGA ESTE CAMPO FALTANTE
            e.get("province"),
            e.get("district"),
            e.get("region"),
            e.get("phone"),
            e.get("gender"),
            e.get("birthday"),
            e.get("university"),
            e.get("degree"),
            e.get("bank"),
            e.get("account_type"),
            e.get("account_number"),
            e.get("nationality"),
            e.get("civil_status"),
            e.get("health_company"),
            e.get("pension_regime"),
            e.get("pension_fund"),
            e.get("active_until"),
            e.get("termination_reason")
        )
        # Verifica que la cantidad de valores sea 38
        if len(values) != 38:
            print(f"⚠️ Error: empleado {e.get('id', 'N/A')} tiene {len(values)} valores, se esperaban 38.")
            print("Valores:", values)
            continue  # Salta este registro
        cursor.execute(sql, values)
        contador += 1
        
        # Mostrar progreso cada 100 registros
        if contador % 100 == 0:
            print(f"📝 Procesados {contador} empleados...")
            
    except Exception as error:
        print(f"⚠️ Error insertando empleado {e.get('id', 'N/A')}: {error}")
        errores += 1

conexion.commit()
print(f"✅ {contador} registros insertados/actualizados en MySQL.")
if errores > 0:
    print(f"⚠️ Se produjeron {errores} errores durante la inserción.")

🚀 Insertando TODOS los empleados en la tabla SQL...
⚠️ Error: empleado 4232 tiene 37 valores, se esperaban 38.
Valores: (4232, 4232, 'Cristian Transito Abarca Garrido', '12.357.791-4', 'abarcacristian0@gmail.com', 'abarcacristian0@gmail.com', '2020-02-04', 'activo', 'Transferencia Bancaria', 4678, '11.524.484-1', 'Indefinido', '2025-04-01', None, None, None, 'Pasaje Boston 1180', '', '', '', 'Maipú', 'RM: Metropolitana de Santiago', '991294377', 'M', '1973-02-27', None, None, 'BCI', 'Corriente', '53527844', 'Chilena', 'Soltero', 'consalud', 'afp', 'modelo', None, None)
⚠️ Error: empleado 4846 tiene 37 valores, se esperaban 38.
Valores: (4827, 4846, 'Javiera Valentina Abarca Riveros', '19.792.744-5', 'javiera.abarca@hotmail.com', 'javiera.abarca@hotmail.com', '2022-04-11', 'activo', 'Transferencia Bancaria', 4514, '18.567.094-5', 'Indefinido', '2025-04-01', None, None, None, 'Central Nino Garcia 3132', '', '', 'Santiago', 'Pedro Aguirre Cerda', 'RM: Metropolitana de Santiago', '98784122

In [ ]:
cursor.close()
conexion.close()
print("✅ Conexión cerrada correctamente.")

In [ ]:
def obtener_todos_los_empleados():
    """
    Función que obtiene los empleados limitando la búsqueda a 4 páginas.
    """
    headers = {"auth_token": TOKEN}
    todos_los_empleados = []
    url_actual = URL
    pagina_actual = 1
    
    print("🚀 Comenzando la obtención de empleados con paginación...")
    
    while url_actual:
        print(f"📄 Obteniendo página {pagina_actual}...")
        
        try:
            respuesta = requests.get(url_actual, headers=headers)
            respuesta.raise_for_status()  # Lanza una excepción para errores HTTP
            
            respuesta_api = respuesta.json()
            empleados_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']
            
            # Agregar empleados de esta página a la lista total
            todos_los_empleados.extend(empleados_pagina)
            
            print(f"✅ Página {pagina_actual}: {len(empleados_pagina)} empleados obtenidos")
            print(f"📊 Total acumulado: {len(todos_los_empleados)} empleados")
            
            # Obtener la URL de la siguiente página
            url_actual = pagination_info.get('next')
            pagina_actual += 1
            
            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la petición: {e}")
            break # Salir del bucle si hay un error
            
    print(f"🎉 ¡Paginación completada! Total de empleados obtenidos: {len(todos_los_empleados)}")
    return todos_los_empleados

# Obtener todos los empleados
empleados = obtener_todos_los_empleados()

🚀 Comenzando la obtención de empleados con paginación...
📄 Obteniendo página 1...
✅ Página 1: 25 empleados obtenidos
📊 Total acumulado: 25 empleados
📄 Obteniendo página 2...
✅ Página 2: 25 empleados obtenidos
📊 Total acumulado: 50 empleados
📄 Obteniendo página 3...
✅ Página 3: 25 empleados obtenidos
📊 Total acumulado: 75 empleados
📄 Obteniendo página 4...
✅ Página 4: 25 empleados obtenidos
📊 Total acumulado: 100 empleados
🎉 ¡Paginación completada! Total de empleados obtenidos: 100


In [ ]:
#
def obtener_todos_los_empleados():
    """
    Función que obtiene todos los empleados iterando por todas las páginas
    """
    headers = {"auth_token": TOKEN}
    todos_los_empleados = []
    url_actual = URL
    pagina_actual = 1
    
    print("🚀 Comenzando la obtención de empleados con paginación...")
    
    while url_actual and pagina_actual <= 5:
        print(f"📄 Obteniendo página {pagina_actual}...")
        
        try:
            respuesta = requests.get(url_actual, headers=headers)
            respuesta.raise_for_status()  # Lanza una excepción para errores HTTP
            #respuesta.status_code == 200:

            respuesta_api = respuesta.json()
            empleados_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']
            
            # Agregar empleados de esta página a la lista total
            todos_los_empleados.extend(empleados_pagina)
            
            print(f"✅ Página {pagina_actual}: {len(empleados_pagina)} empleados obtenidos")
            print(f"📊 Total acumulado: {len(todos_los_empleados)} empleados")
            print(f"📈 Páginas restantes: {pagination_info['total_pages'] - pagina_actual}")
            
            # Verificar si hay una página siguiente
            url_actual = pagination_info.get("previous: 1")
            pagina_actual += 1
      
            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)
        
        else:
             print(f"❌ Error en la petición página {pagina_actual}:", respuesta.status_code)    
        break
    print(f"🎉 ¡Paginación completada! Total de empleados obtenidos: {len(todos_los_empleados)}")
    return todos_los_empleados

# Obtener todos los empleados
empleados = obtener_todos_los_empleados()

SyntaxError: expected 'except' or 'finally' block (4140992005.py, line 38)

In [65]:
print (empleados)

[{'person_id': 4232, 'id': 4232, 'picture_url': 'https://bukwebapp-enterprise-chile.s3.amazonaws.com/cramer/person/picture_url/4232/544f7e72-106d-4246-a779-adfd6adb7790-123577914.jpg.jpg', 'first_name': 'Cristian Transito', 'surname': 'Abarca', 'second_surname': 'Garrido', 'full_name': 'Cristian Transito Abarca Garrido', 'document_type': 'rut', 'document_number': '12.357.791-4', 'rut': '12.357.791-4', 'code_sheet': 'F1', 'email': 'abarcacristian0@gmail.com', 'personal_email': 'abarcacristian0@gmail.com', 'address': 'Pasaje Boston 1180', 'street': '', 'street_number': '', 'office_number': '', 'city': '', 'district': 'Maipú', 'location_id': 334, 'region': 'RM: Metropolitana de Santiago', 'office_phone': '', 'phone': '991294377', 'gender': 'M', 'birthday': '1973-02-27', 'university': None, 'degree': None, 'active_since': '2020-02-04', 'created_at': '2021-10-19T20:59:28.118-03:00', 'status': 'activo', 'payment_method': 'Transferencia Bancaria', 'payment_period': None, 'advance_payment': No

In [88]:
print("🚀 Conectando a MySQL...")
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    #charset='utf8mb4'
)
cursor = conexion.cursor()

# Crear base de datos si no existe
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")


print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")

print("🚀 Creando tabla si no existe...")
sql_create_table = """
CREATE TABLE IF NOT EXISTS empleados (
    id INT PRIMARY KEY,
    first_name VARCHAR(255),
    surname VARCHAR(255),
    full_name VARCHAR(255),
    rut VARCHAR(50),
    email VARCHAR(255),
    phone VARCHAR(50)
);
"""
cursor.execute(sql_create_table)


🚀 Conectando a MySQL...
✅ Conectado a MySQL y usando la base: datos_buk_rrhh
🚀 Creando tabla si no existe...


0

In [89]:
print("🚀 Insertando TODOS los empleados en la tabla SQL...")
contador = 0
errores = 0

for e in empleados:
    try:
        sql = """
        INSERT INTO empleados (id, first_name, surname, full_name, rut, email, phone)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            first_name=VALUES(first_name),
            surname=VALUES(surname),
            full_name=VALUES(full_name),
            rut=VALUES(rut),
            email=VALUES(email),
            phone=VALUES(phone)
        """
        values = (
            e.get("id"),
            e.get("first_name"),
            e.get("surname"),
            e.get("full_name"),
            e.get("rut"),
            e.get("email"),
            e.get("phone")
        )
        cursor.execute(sql, values)
        contador += 1
        
        # Mostrar progreso cada 100 registros
        if contador % 100 == 0:
            print(f"📝 Procesados {contador} empleados...")
            
    except Exception as error:
        print(f"⚠️ Error insertando empleado {e.get('id', 'N/A')}: {error}")
        errores += 1

conexion.commit()
print(f"✅ {contador} registros insertados/actualizados en MySQL.")
if errores > 0:
    print(f"⚠️ Se produjeron {errores} errores durante la inserción.")

🚀 Insertando TODOS los empleados en la tabla SQL...
📝 Procesados 100 empleados...
✅ 100 registros insertados/actualizados en MySQL.


In [90]:
print("🚀 Consultando estadísticas de la tabla...")
cursor.execute("SELECT COUNT(*) as total FROM empleados;")
total_registros = cursor.fetchone()[0]
print(f"📊 Total de empleados en la base de datos: {total_registros}")

print("\n🔍 Mostrando los primeros 5 empleados:")
cursor.execute("SELECT id, full_name, email FROM empleados LIMIT 5;")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Nombre: {row[1]}, Email: {row[2]}")

print("\n🔍 Mostrando los últimos 5 empleados:")
cursor.execute("SELECT id, full_name, email FROM empleados ORDER BY id DESC LIMIT 5;")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Nombre: {row[1]}, Email: {row[2]}")

🚀 Consultando estadísticas de la tabla...
📊 Total de empleados en la base de datos: 100

🔍 Mostrando los primeros 5 empleados:
ID: 4139, Nombre: Flaminio Del Carmen Agurto Urrutia, Email: agurtoflaminio@gmail.com
ID: 4141, Nombre: Jose Aguilar Gonzalez, Email: jfaguilargonza@gmail.com
ID: 4148, Nombre: Pablo Barrales Palma, Email: pablo_barrales@hotmail.com
ID: 4149, Nombre: Claudio Azua Mardones, Email: cazua@cramer.cl
ID: 4201, Nombre: Fabiola Andrea Alfaro Guzman, Email: fabby2410@gmail.com

🔍 Mostrando los últimos 5 empleados:
ID: 11419, Nombre: Nero Angelo Esteban Antihuala Zuñiga, Email: nerozuniga@gmail.com
ID: 11121, Nombre: Giovanni Araya Curín, Email: gi.arayac@gmail.com
ID: 11055, Nombre: Claudio Avaria Reyes, Email: avariaton@gmail.com
ID: 10758, Nombre: Alejandro Andrés Ayala Salas, Email: aa43680045@gmail.com
ID: 10626, Nombre: Jorge Iván Amigo Oliva, Email: jorgeamigooliva@gmail.com


In [ ]:
cursor.close()
conexion.close()
print("✅ Conexión cerrada correctamente.")

✅ Conexión cerrada correctamente.
